# Example: Quantum Convolutional Neural Network

In this notebook we show how to use the `QCNN encoding circuit` for regression and classification.

## Imports

In [ ]:
import numpy as np
from qiskit.circuit import QuantumCircuit
from qiskit.circuit import ParameterVector
from squlearn.encoding_circuit import (
    QiskitEncodingCircuit,
    QCNNEncodingCircuit,
    LayeredEncodingCircuit,
)
from squlearn.qnn import QNNClassifier, QNNRegressor, SquaredLoss
from squlearn.optimizers import Adam
from squlearn import Executor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from squlearn.observables import CustomObservable
from sklearn.datasets import make_moons

## `QCNN encoding circuit` Setup

We start by showing examples of how to build the QCNN. The first example is the most simple way. With the `default` keyword set to "True" the default circuit with the given number of qubits and features is build.

In [ ]:
qc = QCNNEncodingCircuit(num_qubits=4, num_features=4, default=True)
qc.draw("mpl")

Another fast way would be to define a convolution layer and a pooling layer and call the `repeat_layers` function to repeat the first layers until few enough qubits are left in the circuit. The individual building blocks can be resolved with the keyword `decompose`. As one can see, with the keyword `measurement` set to True in the pooling layer one can implement gates on qubits conditioned on the outcome of a measurement on another qubit instead of normal conditional (e.g. cx) gates.

In [ ]:
qc = QCNNEncodingCircuit(4, 2)
qc.convolution(alternating=False)
qc.pooling(measurement=True)
qc.repeat_layers()
qc.draw("mpl", decompose=True)

We can provide our own feature map and add it in front of the encoding circuit. Further we can change the number of qubits and features in the circuit on the fly `up` (and `down`, if the already applied pooling gates allow it).

In [ ]:
qc = QCNNEncodingCircuit(num_qubits=3, default=True)
qc.set_params(num_qubits=4)
encode = LayeredEncodingCircuit.from_string("Rx(x)", num_qubits=4, num_features=2)
qcnn = encode + qc
qcnn.draw("mpl")

We can overwrite the default circuits in the given layer by setting the keyword `quantum_circuit`. The Circuits can either be `QuantumCircuit` or `sQUlearn circuit` objects (if one inputs a circuit with features an Error is raised). Notice that the qiskit `ParameterVector`'s are overwritten with the internal parameters. The circuit size is only restricted by the number of qubits left in the qcnn. In the pooling layer the adressed qubits can be set with the keywords `input_list` and `output_list`.

In [ ]:
qc = QCNNEncodingCircuit(6)

x1 = ParameterVector("p", 2)
qc1 = QuantumCircuit(2)
qc1.crx(x1[0], 0, 1)
qc1.rx(x1[1], 0)

qc2 = QiskitEncodingCircuit(qc1)  # sQUlearn circuit object

x31 = ParameterVector("v", 2)
x32 = ParameterVector("l", 1)
qc3 = QuantumCircuit(3)
qc3.rx(x31[0], 0)
qc3.rx(x31[1], 1)
qc3.rx(x32[0], 2)
qc3.cx(0, 1)
qc3.cx(1, 2)

qc.convolution(quantum_circuit=qc1, label="abi", alternating=False, share_params=True)
qc.pooling(quantum_circuit=qc2, input_list=[[0, 2], [1, 4]], output_list=[[2], [1]])
qc.convolution(quantum_circuit=qc3)
qc.pooling(measurement=True)
qc.fully_connected()
qc.draw("mpl")

Finally we can build the circuit "backwards" by supplying the gates first and then how many qubits should be in the output and you can even continue building from there forward if no fully connected layer is applied. Be careful with adressing the correct qubits when providing in- and output lists.

In [ ]:
# build backwards
qc = QCNNEncodingCircuit()
qc.convolution(alternating=False)
qc.pooling()
qc.repeat_layers(1)
qc.pooling(input_list=[[0, 4]], output_list=[[0]])
qc.build_circuit(2)
# continue forwards
qc.pooling()
qc.fully_connected()
qc.draw("mpl")

The fitting default observable for the circuit can be extracted once finished with building the circuit. Notice that the observable "IIIIIZ" corresponds to only measuring the first qubit "q0".

In [ ]:
obs = qc.QCNNObservable(obs="X")
print(obs)

Alternatively one can provide an Observable for the left qubits which then is mapped onto all qubits.

In [ ]:
qc = QCNNEncodingCircuit(4)
qc.convolution()
qc.pooling()
qc.fully_connected()
obs = CustomObservable(2, ["XY", "ZZ", "ZI", "IY"])
qc.QCNNObservable(obs)

In the following we creat training and test data for a very simple regression problem. This example is supposed to show how to properly work with this encoding circuit in the sQUlearn environment.

In [ ]:
# Data that is inputted to the QNN
X = np.arange(-0.5, 0.6, 0.1).reshape(-1, 1)
# Data that is fitted by the QNN
y = np.square(X).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

Next the encoding circuit and observable are build.

In [ ]:
qc = QCNNEncodingCircuit(4, 1)
qc.convolution()
qc.pooling(measurement=True)
qc.repeat_layers()
qc.fully_connected()

obs = qc.QCNNObservable()
print(obs)
qc.draw("mpl")

Now we can set up the Regressor and train it.

In [ ]:
qnn = QNNRegressor(
    qc,
    obs,
    Executor("pennylane"),
    SquaredLoss(),
    Adam(options={}),
)
qnn.fit(X_train, y_train)

Finally we can plot the results of the regression.

In [ ]:
# ouput optimal parameters
print("Optimal gate parameters:")
print("[")
for i in qnn.param:
    print(str(i) + ",")
print("]")
print("--------------")
print("Optimal observable parameters:")
print("[")
for i in qnn.param_op:
    print(str(i) + ",")
print("]")

y_train_predict = qnn.predict(X_train)
y_test_predict = qnn.predict(X_test)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.set_title("Square function fit")
ax.plot(X, y, label="Goal_function", linestyle="dashed", color="black")
ax.plot(X_train, y_train_predict, label="training predictions", marker="x", linestyle="None")
ax.plot(X_test, y_test_predict, label="test predictions", marker="x", linestyle="None")
ax.set(xlabel="X", ylabel="y")
ax.legend()

sigma_train = np.sqrt(np.sum(np.square(y_train_predict - y_train)))
sigma_test = np.sqrt(np.sum(np.square(y_test_predict - y_test)))
print("sigma train: ", sigma_train, ", sigma test: ", sigma_test)

In the following a classification example is shown. Firstly, data is generated from the sklearn `make_moons` dataset.

In [ ]:
X, y = make_moons(noise=0.3, random_state=1, n_samples=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

plt.scatter(
    [X[i][0] for i in range(len(y)) if y[i] == 1],
    [X[i][1] for i in range(len(y)) if y[i] == 1],
    label="1",
)
plt.scatter(
    [X[i][0] for i in range(len(y)) if y[i] == 0],
    [X[i][1] for i in range(len(y)) if y[i] == 0],
    label="0",
)
plt.legend()
plt.show()

Then we instantiate the QCNN encoding circuit, the Observable and the Classifier.

In [ ]:
qc = QCNNEncodingCircuit(4, 2)
qc.convolution()
qc.pooling(measurement=True)
qc.repeat_layers()
qc.fully_connected()

obs = qc.QCNNObservable()

qnn = QNNClassifier(qc, obs, Executor(), SquaredLoss(), Adam())
print(obs)
qc.draw("mpl")

Finally we train and predict the model.

In [ ]:
# Train the model
qnn.fit(X_train, y_train)

# Data for testing the trained model
print("Training score:", qnn.score(X_train, y_train))
print("Testing score:", qnn.score(X_test, y_test))

The decission surface of the QCNN for the two moons problem is shown.

In [ ]:
X_new = []
for i in range(-20, 21):
    for j in range(-10, 16):
        X_new.append([i / 10, j / 10])
y_pred = qnn.predict(X_new)
plt.scatter(
    [X_new[i][0] for i in range(len(y_pred)) if y_pred[i] == 1],
    [X_new[i][1] for i in range(len(y_pred)) if y_pred[i] == 1],
    label="1",
)
plt.scatter(
    [X_new[i][0] for i in range(len(y_pred)) if y_pred[i] == 0],
    [X_new[i][1] for i in range(len(y_pred)) if y_pred[i] == 0],
    label="0",
)
plt.legend()
plt.show()